In [1]:
import os

%pwd

'/Users/wilsvenleong/Downloads/learning-materials/mlflow/research'

In [2]:
os.chdir("../")  # Move up to root directory
%pwd

'/Users/wilsvenleong/Downloads/learning-materials/mlflow'

## 0. Set Environment Variables in `.env`

In [3]:
from dotenv import load_dotenv


load_dotenv()

MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI")
MLFLOW_TRACKING_USERNAME = os.getenv("MLFLOW_TRACKING_USERNAME")
MLFLOW_TRACKING_PASSWORD = os.getenv("MLFLOW_TRACKING_PASSWORD")

## 1. Update `config.yaml`

```yaml
model_evaluation:
  root_dir: artifacts/model_evaluation
  test_data_path: artifacts/data_transformation/test.csv
  model_path: artifacts/model_trainer/model.joblib
  metrics_file_path: artifacts/model_evaluation/metrics.json
```

## 2. Update `schema.yaml`

```yaml
columns:
  fixed acidity: float64
  volatile acidity: float64
  citric acid: float64
  residual sugar: float64
  chlorides: float64
  free sulfur dioxide: float64
  total sulfur dioxide: float64
  density: float64
  pH: float64
  sulphates: float64
  alcohol: float64
  quality: int64

target:
  name: quality
```

## 4. Update the Entity

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metrics_file_path: Path
    alpha: float
    l1_ratio: float
    random_state: int
    target_column: str
    mlflow_uri: str

## 5. Update the `ConfigurationManager` in `src/config/configuration.py`

In [5]:
from mlflow_project.constants import *
from mlflow_project.utils.common import read_yaml, create_directories

from mlflow_project import logger


class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH,
        schema_file_path: Path = SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        """
        Creates the root directory and returns
        the configuration for model evaluation.

        Returns:
            ModelEvaluationConfig: Configuration for model evaluation.
        """
        model_evaluation = self.config.model_evaluation
        target = self.schema.target

        create_directories([model_evaluation.root_dir])

        file_path = os.path.join(model_evaluation.root_dir, ".gitkeep")
        if not os.path.exists(file_path):
            with open(file_path, "w") as f:
                logger.info(
                    f"Creating file: .gitkeep in directory {model_evaluation.root_dir}"
                )
                pass

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=Path(model_evaluation.root_dir),
            test_data_path=Path(model_evaluation.test_data_path),
            model_path=Path(model_evaluation.model_path),
            metrics_file_path=Path(model_evaluation.metrics_file_path),
            alpha=float(self.params.ALPHA),
            l1_ratio=float(self.params.L1_RATIO),
            random_state=int(self.params.RANDOM_STATE),
            target_column=str(target.name),
            mlflow_uri=str(MLFLOW_TRACKING_URI),
        )

        return model_evaluation_config

## 6. Update the Components

In [6]:
from typing import Union
from urllib.parse import urlparse

import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import mlflow
import mlflow.sklearn

from mlflow_project.utils.common import save_json


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(
        self,
        actual: Union[np.ndarray, pd.DataFrame],
        pred: Union[np.ndarray, pd.DataFrame],
    ) -> tuple[float]:
        """Calculates evaluation metrics.

        Args:
            actual (Union[np.ndarray, pd.DataFrame]): Actual results.
            pred (Union[np.ndarray, pd.DataFrame]): Predicted results.

        Returns:
            tuple(float): The evaluation metrics (i.e. RMSE, MAE and R2) as a tuple of floats.
        """
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)

        return rmse, mae, r2

    def log_into_mlflow(self) -> None:
        """MLFlow for experiment tracking."""
        # Load the test set
        test_data = pd.read_csv(self.config.test_data_path)
        # Split the test set into independent and dependent features
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        # Load the pre-trained model
        model = joblib.load(self.config.model_path)

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            # Predictions
            pred_y = model.predict(test_x)
            # Evaluate metrics
            rmse, mae, r2 = self.eval_metrics(test_y, pred_y)

            # Saving metrics locally
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metrics_file_path), data=scores)

            params = {
                "alpha": self.config.alpha,
                "l1_ratio": self.config.l1_ratio,
                "random_state": self.config.random_state,
            }
            # Log hyperparameters
            mlflow.log_params(params)

            # Log metrics
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # Please refer to the documentation for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(
                    model, "model", registered_model_name="ElasticNetModel"
                )
            else:
                mlflow.sklearn.log_model(model, "model")

## 7. Update the Pipeline

In [7]:
try:
    config_manager = ConfigurationManager()
    model_evaluation_config = config_manager.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    logger.exception(e)
    raise e

[ 2023-10-25 20:33:40,770 ] 33 common mlflow_project -  INFO - Loaded YAML file successfully from: config/config.yaml
[ 2023-10-25 20:33:40,774 ] 33 common mlflow_project -  INFO - Loaded YAML file successfully from: params.yaml
[ 2023-10-25 20:33:40,776 ] 33 common mlflow_project -  INFO - Loaded YAML file successfully from: schema.yaml
[ 2023-10-25 20:33:40,777 ] 53 common mlflow_project -  INFO - Created directory at: artifacts
[ 2023-10-25 20:33:40,778 ] 53 common mlflow_project -  INFO - Created directory at: artifacts/model_evaluation
[ 2023-10-25 20:33:40,778 ] 36 3972561000 mlflow_project -  INFO - Creating file: .gitkeep in directory artifacts/model_evaluation
[ 2023-10-25 20:33:51,881 ] 67 common mlflow_project -  INFO - JSON file saved at: artifacts/model_evaluation/metrics.json


Successfully registered model 'ElasticNetModel'.
2023/10/25 20:34:07 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 1
Created version '1' of model 'ElasticNetModel'.
